# BioExplorer - Neuronmodulation covers
![](../bioexplorer_neurons_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3

url = 'r1i7n31.bbp.epfl.ch:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

In [ ]:
scale = Vector3(1000, 1000, 1000)
varicosity = Vector3(scale.x * 59.711, scale.y * 1085.713, scale.z * 333.107)
synapse = Vector3(scale.x * 75.4, scale.y * 1080.02, scale.z * 320.1)


In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm

db_host = os.getenv('DB_HOST')
db_host = '10.80.7.21'
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = 'o1'

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

### Load neurons

In [ ]:
presynaptic_neuron_id = 47211
neuron_population_name = 'o1'

presynaptic_assembly_name = 'PreSynaptic Neuron'
be.remove_assembly(presynaptic_assembly_name)
neurons_assembly = be.add_assembly(presynaptic_assembly_name)
status = be.add_neurons(
    assembly_name=presynaptic_assembly_name,
    population_name=neuron_population_name,
    geometry_quality=be.GEOMETRY_QUALITY_MEDIUM,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION,
    use_sdf=True, load_synapses=True, generate_varicosities=True,
    load_somas=True, load_axon=True, show_membrane=True,
    load_basal_dendrites=True, load_apical_dendrites=True,
    generate_internals=True, generate_externals=False,
    sql_node_filter='guid=%d' % presynaptic_neuron_id,
    scale=scale
)

In [ ]:
postsynaptic_neuron_id = 49
postsynaptic_assembly_name = 'PostSynaptic Neuron'
be.remove_assembly(postsynaptic_assembly_name)
neurons_assembly = be.add_assembly(postsynaptic_assembly_name)

with Session(engine) as session:
    data = session.execute('SELECT x,y,z FROM %s.node WHERE guid=%d' % (db_schema, postsynaptic_neuron_id))
    soma_position = data.all()[0]

status = be.add_neurons(
    assembly_name=postsynaptic_assembly_name,
    population_name=neuron_population_name,
    geometry_quality=be.GEOMETRY_QUALITY_MEDIUM,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION,
    use_sdf=True, load_synapses=True, generate_varicosities=True,
    load_somas=True, load_axon=True, show_membrane=True,
    load_basal_dendrites=True, load_apical_dendrites=True,
    generate_internals=True, generate_externals=False,
    sql_node_filter='guid=%d' % postsynaptic_neuron_id,
    scale=scale
)

model_ids = be.get_model_ids()['ids']
model_id = model_ids[len(model_ids)-1]
tf = {
    'rotation': [0.0, 0.0, 0.0, 1.0], 
    'rotation_center': [0.0, 0.0, 0.0], 
    'scale': [1.0, 1.0, 1.0], 
    'translation': [
        scale.x * (varicosity.x / scale.x - soma_position[0] - 0),
        scale.y * (varicosity.y / scale.y - soma_position[1] - 0),
        scale.z * (varicosity.z / scale.z - soma_position[2] - 7),
    ]
}
core.update_model(id=model_id, transformation=tf)

In [ ]:
presynaptic_neuron_id = 47211
neuron_population_name = 'o1'

presynaptic_assembly_name = 'PreSynaptic Neuron 2'
be.remove_assembly(presynaptic_assembly_name)
neurons_assembly = be.add_assembly(presynaptic_assembly_name)
status = be.add_neurons(
    assembly_name=presynaptic_assembly_name,
    population_name=neuron_population_name,
    geometry_quality=be.GEOMETRY_QUALITY_MEDIUM,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION,
    use_sdf=True, load_synapses=True, generate_varicosities=True,
    load_somas=True, load_axon=True, show_membrane=True,
    load_basal_dendrites=True, load_apical_dendrites=True,
    generate_internals=True, generate_externals=False,
    sql_node_filter='guid=%d' % presynaptic_neuron_id,
    scale=scale
)

model_ids = be.get_model_ids()['ids']
model_id = model_ids[len(model_ids)-1]
tf = {
    'rotation': [0.0, 0.0, 0.0, 1.0], 
    'rotation_center': [0.0, 0.0, 0.0], 
    'scale': [1.0, 1.0, 1.0], 
    'translation': [scale.x * 13.350, scale.y * 5.750, scale.z * 2.850],
}
core.update_model(id=model_id, transformation=tf)

In [ ]:
vasculature_assembly_name = 'Vasculature'
vasculature_population_name = 'vasculature'
be.remove_assembly(vasculature_assembly_name)
vasculature_assembly = be.add_assembly(vasculature_assembly_name)
vasculature_model = be.add_vasculature(
    assembly_name=vasculature_assembly_name,
    population_name=vasculature_population_name,
    use_sdf=True,
    sql_filter='sqrt(pow(x - %f, 2) + pow(y - %f, 2) + pow(z - %f, 2)) < 200' % (varicosity.x / scale.x, varicosity.y / scale.y, varicosity.z / scale.z),
    scale=scale
)

In [ ]:
astrocytes_assembly_name = 'Astrocytes'
be.remove_assembly(astrocytes_assembly_name)
astrocytes_assembly = be.add_assembly(astrocytes_assembly_name)
astrocytes_model = be.add_astrocytes(
    assembly_name=astrocytes_assembly_name,
    population_name=vasculature_population_name,
    vasculature_population_name=vasculature_population_name,
    radius_multiplier=0.5,
    use_sdf=True, generate_internals=True,
    load_somas=True, load_dendrites=True,
    sql_filter='sqrt(pow(x - %f, 2) + pow(y - %f, 2) + pow(z - %f, 2)) < 200 AND guid%%3=1' % (varicosity.x / scale.x, varicosity.y / scale.y, varicosity.z / scale.z),
    scale=scale)


In [ ]:
from bioexplorer import Protein, Volume, AnimationParams

resource_folder = '../../tests/test_files/'
pdb_folder = os.path.join(resource_folder, 'pdb')
acetylcholin_path = os.path.join(pdb_folder, 'neuromodulation', 'acetylcholin.pdb')

def add_acetylcholin(position, radius, nb_molecules, radius_multiplier, frame=0):
    protein_representation = be.REPRESENTATION_ATOMS

    acetylcholin_assembly_name = 'Acetylcholin'
    acetylcholin_name = 'Acetylcholin'
    
    acetylcholin = Protein(
        name=acetylcholin_name, 
        source=acetylcholin_path,
        load_non_polymer_chemicals=True, load_bonds=True, load_hydrogen=True,
        occurences=nb_molecules,
        animation_params=AnimationParams(3, (frame + 1) * 2, 0.5, (frame + 2) * 2, 1.0)
    )
    volume = Volume(
        name=acetylcholin_assembly_name,
        shape=be.ASSEMBLY_SHAPE_FILLED_SPHERE,
        shape_params=Vector3(radius, 0.0, 0.0),
        protein=acetylcholin)
    be.add_volume(
        volume=volume, 
        representation=protein_representation,
        position=position,
        atom_radius_multiplier=radius_multiplier)


In [ ]:
# add_acetylcholin(varicosity, scale.x * 1.0, 50000, 30.0)
# add_acetylcholin(synapse, scale.x * 0.2, 500000, 2.0)
# core.set_renderer()

In [ ]:
core.set_camera(
    orientation=[0.053963664939095296, 0.8868302250065068, 0.15284292066531585, 0.4327344641737535],
    position=[scale.x * 78.857, scale.y * 1081.082, scale.z * 317.393],
    target=[scale.x * 75.576, scale.y * 1080.143, scale.z * 319.814]    
)

### Materials

In [ ]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name, palette_size, shading_mode, glossiness=1.0, emission=0.0, user_param=1.0):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette = sns.color_palette(palette_name, palette_size)

    for material_id in material_ids:
        mid = material_id % palette_size
        if mid in [be.NEURON_MATERIAL_AFFERENT_SYNPASE, be.NEURON_MATERIAL_EFFERENT_SYNPASE]:
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(5.0)
        elif mid == be.NEURON_MATERIAL_MITOCHONDRION:
            colors.append([0.5, 0.1, 0.6])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(6.0)
        elif mid == be.NEURON_MATERIAL_NUCLEUS:
            colors.append([1.0, 1.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(30.0)
        elif mid == be.NEURON_MATERIAL_SOMA:
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(5.0)
        elif mid == be.NEURON_MATERIAL_MYELIN_STEATH:
            colors.append([0.4, 0.3, 0.5])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(50.0)
        else:
            # Membrane
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(5.0)
            
        refraction_indices.append(1.0)
        emissions.append(emission)
        user_params.append(user_param)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

In [ ]:
def set_materials():
    palettes = ['GnBu_r', 'PuRd', 'GnBu_r', 'Set1', 'Wistia', 'Greys']
    model_ids = be.get_model_ids()['ids']
    i = 0
    for model_id in model_ids:
        emission = 0.0
        if i==5:
            continue
        set_morphology_materials(
            model_id, palettes[i], be.NB_MATERIALS_PER_MORPHOLOGY,
            be.SHADING_MODE_PERLIN, 0.1, emission, 0.1 / scale.x)
            # be.SHADING_MODE_CARTOON, 1.0, emission, 3.0)
        i += 1
    core.set_renderer()

set_materials()

In [ ]:
status = core.set_renderer(
    background_color=[0.18, 0.43, 0.41],
    current='bio_explorer',subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.fog_start = 1.0 * scale.x
params.fog_thickness = 300.0 * scale.x
params.gi_samples = 1
params.gi_weight = 0.2
params.gi_distance = 1.0 * scale.x
params.shadows = 1.0
params.soft_shadows = 1.0
params.epsilon_factor = 100.0
params.max_bounces = 3
params.show_background = False
params.use_hardware_randomizer=True
status = core.set_renderer_params(params)
status = core.set_renderer()

In [ ]:
core.set_camera(
    orientation=[-0.070, 0.437, 0.220, 0.868],
    position=[scale.x * 77.98042386871596, scale.y * 1091.834739623779, scale.z * 348.5927433943219],
    target=[scale.x * 13.78747115930694, scale.y * 1064.041932405726, scale.z * 295.22463810933783]
)

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

In [ ]:
stop

## Snapshots

In [ ]:
core.set_camera(
    orientation=[0.10042722106643472, 0.5480670250715927, 0.10408422725871022, 0.823834560414896],
    position=[70884.91927496702, 1086726.4432583167, 338806.9308694824],
    target=[2906.5292038353564, 1090506.7453414442, 310916.74450726254]
)
mm.create_snapshot(
    renderer='bio_explorer',
    size=[3840, 2160], samples_per_pixel=64,
    base_name='neuromodulation_4k_v7',
    path='/gpfs/bbp.cscs.ch/project/proj129/images/neuromodulation')

In [ ]:
add_acetylcholin(scale.x * 2.0, 10000000, 1.0)

In [ ]:
core.set_camera(
    orientation=[0.11414384254134172, 0.3981421943009997, 0.08577072175808344, 0.9061442267190032],
    position=[60426.19242247517, 1086103.9795371932, 336060.44751665404],
    target=[15423.078123496032, 1094517.896698822, 296171.4300538681]
)

mm.create_snapshot(
    renderer='bio_explorer',
    size=[3840, 2160], samples_per_pixel=64,
    base_name='neuromodulation_4k_v8',
    path='/gpfs/bbp.cscs.ch/project/proj129/images/neuromodulation')

In [ ]:
core.set_camera(
    orientation=[-0.08616460361504102, 0.5168722715162748, 0.2108410708440709,0.8252058887737016],
    position=[73092.67928336751, 1091013.0323442845, 339637.27257494023],
    target=[9012.827321822671, 1062754.583344163, 304264.48103967734],    
)

mm.create_snapshot(
    renderer='bio_explorer',
    size=[3840, 2160], samples_per_pixel=64,
    base_name='neuromodulation_4k_v9',
    path='/gpfs/bbp.cscs.ch/project/proj129/images/neuromodulation')

In [ ]:
core.set_camera(
    orientation=[0.11168463198861378, 0.4756995788056184, 0.09304005355047734, 0.8675136898845167],
    position=[93057.10494369318, 1081509.836538513, 352380.7404783915],
    target=[11963.415460410512, 1091597.755994066, 302306.8141172933]
)

mm.create_snapshot(
    renderer='bio_explorer',
    size=[3840, 2160], samples_per_pixel=64,
    base_name='neuromodulation_4k_v10',
    path='/gpfs/bbp.cscs.ch/project/proj129/images/neuromodulation')

## Movie

In [ ]:
key_frames = [
    {
        'apertureRadius': 0.0,
        'direction': [-0.3309065586477476, -0.5515454842588424, -0.7657012656627804],
        'focusDistance': 1000000.0,
        'origin': [85642.08338486325, 1129925.2832373616, 385832.9375390083],
        'up': [-0.6899312727984696, 0.6949887822126584, -0.20244858955608871]
    },    
    {
        'apertureRadius': 0.0,
        'direction': [-0.3494389545912042, -0.5500107228415851, -0.7585384774508697],
        'focusDistance': 1000000.0,
        'origin': [64618.180875309445, 1090766.3611805325, 336370.7998428331],
        'up': [-0.6840261380411214, 0.703011566021698, -0.19463550682305786]
    },
    {
        'apertureRadius': 0.0,
        'direction': [-0.921620789460506, -0.3707483460733247, -0.1147204616364843],
        'focusDistance': 1000000.0,
        'origin': [61916.89367761961, 1086689.9173568897, 332985.8536498305],
        'up': [-0.38164014454181666, 0.9194737851371254, 0.0944391789444855]
    },
    {
        'apertureRadius': 0.0,
        'direction': [-0.921620789460506, -0.3707483460733247, -0.1147204616364843],
        'focusDistance': 1000000.0,
        'origin': [61916.89367761961, 1086689.9173568897, 332985.8536498305],
        'up': [-0.38164014454181666, 0.9194737851371254, 0.0944391789444855]
    },
    {
        'apertureRadius': 0.0,
        'direction': [-0.8371737098698813, -0.37502167711053114, -0.39811922987957327],
        'focusDistance': 1000000.0,
        'origin': [68429.12995521477, 1089034.4591921489, 336340.9125636818],
        'up': [-0.42932080875343503, 0.9015645158280192, 0.05352632035817306]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.365843687075761, -0.5477563005205924, -0.7524104145124568],
        'focusDistance': 1000000.0,
        'origin': [65261.41516267728, 1093698.5987370815, 341342.6803248807],
        'up': [0.16890046903040096, 0.8341071962457943, -0.5251074335146003]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.5915181593585027, -0.3569236266916338, -0.7229880993892084],
        'focusDistance': 1000000.0,
        'origin': [74398.38101087196, 1080512.3579775202, 321014.29011693463],
        'up': [0.3708437572822547, 0.9166451523800699, -0.14911932236601122]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.5915181593376176, -0.35692362683095524, -0.722988099337516],
        'focusDistance': 1000000.0,
        'origin': [73509.77975947048, 1081022.511364776, 320994.49975719076],
        'up': [0.37084375736693764, 0.9166451523289721, -0.14911932246951562]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.5915181593376176, -0.35692362683095524, -0.722988099337516],
        'focusDistance': 1000000.0,
        'origin': [73509.77975947048, 1081022.511364776, 320994.49975719076],
        'up': [0.37084375736693764, 0.9166451523289721, -0.14911932246951562]
    }
 ]
 

mm.build_camera_path(key_frames, 100, 100)

In [ ]:
from IPython.display import clear_output

draft = False
image_size = [1920, 1080]


atom_radius_multiplier = 2.0
diffusion_1_nb_molecules = 5000000
diffusion_1_start_frame = 150
diffusion_1_nb_frames = 325

diffusion_2_nb_molecules = 500000
diffusion_2_start_frame = 600
diffusion_2_nb_frames = 200

nb_frames = mm.get_nb_frames()
for frame in range(nb_frames):
    clear_output()
    print('Frame %d/%d' % (frame, nb_frames))
    if frame >= diffusion_1_start_frame and frame<diffusion_1_start_frame + diffusion_1_nb_frames:
        add_acetylcholin(
            varicosity, scale.x * (0.88 + (frame - diffusion_1_start_frame) / diffusion_1_nb_frames),
            diffusion_1_nb_molecules, atom_radius_multiplier, frame)
    elif frame >= diffusion_2_start_frame and frame<diffusion_2_start_frame + diffusion_2_nb_frames:
        add_acetylcholin(
            synapse, scale.x * (0.1 + (frame - diffusion_2_start_frame) / (3* diffusion_2_nb_frames)),
            diffusion_2_nb_molecules, 10 * atom_radius_multiplier, frame)

    mm.set_current_frame(frame)
    if draft:
        core.set_renderer()
        import time
        time.sleep(0.1)
    else:
        mm.create_snapshot(
            renderer='depth',
            size=image_size, samples_per_pixel=2,
            base_name='%05d' % frame,
            path='/gpfs/bbp.cscs.ch/project/proj129/movies/neuromodulation/v2/depth')
        mm.create_snapshot(
            renderer='bio_explorer',
            size=image_size, samples_per_pixel=64,
            base_name='%05d' % frame,
            path='/gpfs/bbp.cscs.ch/project/proj129/movies/neuromodulation/v2/bio_explorer')
